In [14]:
import numpy as np
import sys
import os
#sys.path.remove('/opt/ros/kinetic/lib/python2.7/dist-packages')
import glob
import cv2
import matplotlib.pyplot as plt
import pandas
import imutils
from tqdm import tqdm
from cv2 import VideoWriter
global rect
from PIL import Image as im
import math
import cv2
print(cv2.__version__)

#1st vehicle - RED
v1_lower_bound = np.array([0, 0, 0])
v1_upper_bound = np.array([65, 255, 255])
#2nd vehicle - BLUE
v2_lower_bound = np.array([0, 0, 0])
v2_upper_bound = np.array([15, 0, 100])
#3rd vehicle - GREEN
v3_lower_bound = np.array([10, 110, 20])
v3_upper_bound = np.array([90, 250, 180])

#BGR colors
bgr_red = (50, 50, 255)
bgr_blue = (255, 175, 70)
bgr_green = (0, 255, 0)


#variables to compute velocity, acceleration
prev_ftime = "0"

v1_x_prev_ref = 0
v1_y_prev_ref = 0
v1_prev_velocity = 0.0

v2_x_prev_ref = 0
v2_y_prev_ref = 0
v2_prev_velocity = 0.0

v3_x_prev_ref = 0
v3_y_prev_ref = 0
v3_prev_velocity = 0.0

#conversion - 60 pixel = 28 cm
pixel_to_meter = 28 / 60 / 100


fourcc = cv2.VideoWriter_fourcc(*'FMP4')
width = 1280
height = 720
FPS = 25 # was 20
seconds = 100
video = VideoWriter('video_FOV.avi', fourcc, float(FPS), (width, height))

def image_resize(image, width = None, height = None, inter = cv2.INTER_AREA):
    # initialize the dimensions of the image to be resized and
    # grab the image size
    dim = None
    (h, w) = image.shape[:2]

    # if both the width and height are None, then return the
    # original image
    if width is None and height is None:
        return image

    # check to see if the width is None
    if width is None:
        # calculate the ratio of the height and construct the
        # dimensions
        r = height / float(h)
        dim = (int(w * r), height)

    # otherwise, the height is None
    else:
        # calculate the ratio of the width and construct the
        # dimensions
        r = width / float(w)
        dim = (width, int(h * r))

    # resize the image
    resized = cv2.resize(image, dim, interpolation = inter)

    # return the resized image
    return resized
    

def get_coordinates(img, interval, frameDelta, prev_velocity, x_prev_ref, y_prev_ref, x_prev, y_prev, x_ref, y_ref, lower_bound, upper_bound, color, color_code, text_displacement, img_overlay):   
    final_x = 0;
    final_y = 0;
    final_yaw = 0;
    image = img 
    backSub = cv2.createBackgroundSubtractorMOG2()
    fgMask = backSub.apply(frameDelta)
    (thresh, mask) = cv2.threshold(fgMask, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    mask = cv2.bitwise_not(mask)
    img = cv2.bitwise_and(img,img,mask = mask)
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, thresh = cv2.threshold(img_gray, 10,255,0)
    num_labels, labels = cv2.connectedComponents(thresh)
    for a in range(num_labels-1):
        masks = np.zeros(thresh.shape, dtype=np.uint8)
        masks[labels == (a+1)] = 255
        cnts = cv2.findContours(masks,cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)   
        cnts = cnts[0] if len(cnts) == 2 else cnts[1]
        # compute the bounding box for the contour, draw it on the frame,
        rect = cv2.minAreaRect(cnts[0])
        box = cv2.boxPoints(rect)
        box = np.int0(box)
        x = int(rect[0][0])
        y = int(rect[0][1])
        b_counter, g_counter, r_counter = 0, 0, 0
        for a in range(5):
            blue, green, red = image[y+a][x+a]
            b_counter = blue + b_counter
            g_counter = green + g_counter
            r_counter = red + r_counter 
            blue, green, red = image[y-a][x-a]
            b_counter = blue + b_counter
            g_counter = green + g_counter
            r_counter = red + r_counter 
        blue, green, red = b_counter/10, g_counter/10, r_counter/10    
        
        if (color == "BLUE" and blue > red and blue > green) or (color == "RED" and red > blue and red > green) or (color == "GREEN" and green > blue and green > red):

            horizontal = np.array([1,0])
            p0 = box[0]
            p1 = box[1] - box[0]
            p2 = box[2] - box[0]
            p3 = box[3] - box[0]

            sum1 = np.sum(p1**2)
            sum2 = np.sum(p2**2)
            sum3 = np.sum(p3**2)

            length1 = np.sqrt(sum1)
            length2 = np.sqrt(sum2)
            length3 = np.sqrt(sum3)

            list123 = [length1, length2, length3]
            mid_length = np.median(list123)

            if mid_length == length1:
                if y > 360:
                    if box[1][0] > box[0][0]:
                        yaw_axis = p1
                        p_start = box[0]
                        p_end = box[1]
                    elif box[1][0] < box[0][0]:
                        yaw_axis = box[0] - box[1]
                        p_start = box[1]
                        p_end = box[0]
                    else:
                        if x > 455:
                            if box[1][1] > box[0][1]:
                                yaw_axis = box[0] - box[1]
                                p_start = box[1]
                                p_end = box[0]
                            elif box[1][1] < box[0][1]:
                                yaw_axis = p1
                                p_start = box[0]
                                p_end = box[1]
                        else:
                            if box[1][1] > box[0][1]:
                                yaw_axis = p1
                                p_start = box[0]
                                p_end = box[1]
                            elif box[1][1] < box[0][1]:
                                yaw_axis = box[0] - box[1]
                                p_start = box[1]
                                p_end = box[0]
                else:
                    if box[1][0] > box[0][0]:
                        yaw_axis = box[0] - box[1]
                        p_start = box[1]
                        p_end = box[0]
                    elif box[1][0] < box[0][0]:
                        yaw_axis = p1
                        p_start = box[0]
                        p_end = box[1]
                    else:
                        if x > 455:
                            if box[1][1] > box[0][1]:
                                yaw_axis = box[0] - box[1]
                                p_start = box[1]
                                p_end = box[0]
                            elif box[1][1] < box[0][1]:
                                yaw_axis = p1
                                p_start = box[0]
                                p_end = box[1]
                        else:
                            if box[1][1] > box[0][1]:
                                yaw_axis = p1
                                p_start = box[0]
                                p_end = box[1]
                            elif box[1][1] < box[0][1]:
                                yaw_axis = box[0] - box[1]
                                p_start = box[1]
                                p_end = box[0]

            elif mid_length == length2:
                if y > 360:
                    if box[2][0] > box[0][0]:
                        yaw_axis = p2
                        p_start = box[0]
                        p_end = box[2]
                    elif box[2][0] < box[0][0]:
                        yaw_axis = box[0] - box[2]
                        p_start = box[2]
                        p_end = box[0]
                    else:
                        if x > 455:
                            if box[2][1] > box[0][1]:
                                yaw_axis = box[0] - box[2]
                                p_start = box[2]
                                p_end = box[0]
                            elif box[2][1] < box[0][1]:
                                yaw_axis = p2
                                p_start = box[0]
                                p_end = box[2]
                        else:
                            if box[2][1] > box[0][1]:
                                yaw_axis = p2
                                p_start = box[0]
                                p_end = box[2]
                            elif box[2][1] < box[0][1]:
                                yaw_axis = box[0] - box[2]
                                p_start = box[2]
                                p_end = box[0]
                else:
                    if box[2][0] > box[0][0]:
                        yaw_axis = box[0] - box[2]
                        p_start = box[2]
                        p_end = box[0]
                    elif box[2][0] < box[0][0]:
                        yaw_axis = p2
                        p_start = box[0]
                        p_end = box[2]
                    else:
                        if x > 455:
                            if box[2][1] > box[0][1]:
                                yaw_axis = box[0] - box[2]
                                p_start = box[2]
                                p_end = box[0]
                            elif box[2][1] < box[0][1]:
                                yaw_axis = p2
                                p_start = box[0]
                                p_end = box[2]
                        else:
                            if box[2][1] > box[0][1]:
                                yaw_axis = p2
                                p_start = box[0]
                                p_end = box[2]
                            elif box[2][1] < box[0][1]:
                                yaw_axis = box[0] - box[2]
                                p_start = box[2]
                                p_end = box[0]

            elif mid_length == length3:
                if y > 360:
                    if box[3][0] > box[0][0]:
                        yaw_axis = p3
                        p_start = box[0]
                        p_end = box[3]
                    elif box[3][0] < box[0][0]:
                        yaw_axis = box[0] - box[3]
                        p_start = box[3]
                        p_end = box[0]
                    else:
                        if x > 455:
                            if box[3][1] > box[0][1]:
                                yaw_axis = box[0] - box[3]
                                p_start = box[3]
                                p_end = box[0]
                            elif box[3][1] < box[0][1]:
                                yaw_axis = p3
                                p_start = box[0]
                                p_end = box[3]
                        else:
                            if box[3][1] > box[0][1]:
                                yaw_axis = p3
                                p_start = box[0]
                                p_end = box[3]
                            elif box[3][1] < box[0][1]:
                                yaw_axis = box[0] - box[3]
                                p_start = box[3]
                                p_end = box[0]
                else:
                    if box[3][0] > box[0][0]:
                        yaw_axis = box[0] - box[3]
                        p_start = box[3]
                        p_end = box[0]
                    elif box[3][0] < box[0][0]:
                        yaw_axis = p3
                        p_start = box[0]
                        p_end = box[3]
                    else:
                        if x > 455:
                            if box[3][1] > box[0][1]:
                                yaw_axis = box[0] - box[3]
                                p_start = box[3]
                                p_end = box[0]
                            elif box[3][1] < box[0][1]:
                                yaw_axis = p3
                                p_start = box[0]
                                p_end = box[3]
                        else:
                            if box[3][1] > box[0][1]:
                                yaw_axis = p3
                                p_start = box[0]
                                p_end = box[3]
                            elif box[3][1] < box[0][1]:
                                yaw_axis = box[0] - box[3]
                                p_start = box[3]
                                p_end = box[0]

            angle = np.arctan2(yaw_axis[1], yaw_axis[0])
            yaw = -int(np.degrees(angle))     
            if yaw < 0:
                yaw += 360

            cv2.drawContours(image, [box], 0, (0, 255, 0), 3)  
            #cv2.arrowedLine(image, ((p_start[0]), (p_start[1])), (p_end[0], p_end[1]), (0, 0, 255), 3)


            cv2.putText(image, "{}".format(color), (70 + text_displacement, 240), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color_code, 2)
            text = "[Center: (" + str('{:.1f}'.format((x-x_ref)*pixel_to_meter)) + "m," + str('{:.1f}'.format((y_ref-y)*pixel_to_meter)) + "m)]"        
            cv2.putText(image, "{}".format(text), (70 + text_displacement, 260), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color_code, 2)
            text = "[Yaw: " +  str(yaw) + " degree]"        
            cv2.putText(image, "{}".format(text), (70 + text_displacement, 280), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color_code, 2)
            final_x = int(rect[0][0])
            final_y = int(rect[0][1])
            final_yaw = yaw

            #reference frame is printed on every function call - need to optimize
            text = "Reference Frame"
            cv2.putText(image, "{}".format(text), (x_ref+70, y_ref-25), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            cv2.line(image, (x_ref,y_ref), (x_ref, y_ref-20), (0,255,0), 3)
            cv2.line(image, (x_ref,y_ref), (x_ref+50, y_ref), (0,255,0), 3)
            text = "0 degree"
            cv2.putText(image, "{}".format(text), (x_ref+80, y_ref-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)


            #draw yellow ellipse
            #cv2.ellipse(img_overlay, (x, y), (75,50), -yaw, -90, 90, (0, 255, 255), -1)        

            #finding distance with pythagorean theorem
            distance = math.sqrt(((x-x_ref-x_prev_ref)**2) + ((y_ref-y-y_prev_ref)**2))
            velocity = ((distance * pixel_to_meter) / interval)
            text = "[Velocity.: " + str('{:.1f}'.format(velocity*3600/1000)) + " km/h]"        
            cv2.putText(image, "{}".format(text), (70 + text_displacement, 300), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color_code, 2)  

            #text = "[Acc.: " + str('{:+.1f}'.format(acceleration)) + " m/s^2]"
            #cv2.putText(image, "{}".format(text), (70 + text_displacement, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color_code, 2)

            #set variable values to compute for next iteration
            x_prev_ref = (x-x_ref)
            y_prev_ref = (y_ref-y)
            prev_velocity = velocity
        
    # Return the coordinates
    return final_x, final_y, final_yaw, image, prev_velocity, x_prev_ref, y_prev_ref, img_overlay

def get_interval(ftime):
    
    global prev_ftime
    #instead of using constant value 17, use space as buffer for substring
    ftime = ftime[17:-1]
    #inverval of two frames for velocity
    print(ftime)
    interval = float(ftime) - float(prev_ftime)
    prev_ftime = ftime
    return interval
    
def get_ref_coordinates(background_path):
    #get reference frame coordinates based on bottom-most and left-most cone 
    img = im.open(background_path)
    w, h = img.size
    pix = img.load()
    x_ref, y_ref = 0, 0
    #iterables chosen such that loop only covers bottom left section of background image
    for i in range(int(3*h/4),h):
        for j in range(int(w/4)):
            r, g, b, *A = pix[j,i]
            #if statement includes approximated rgb ranges for possible shades of orange
            if r >= 200 and 110 <= g <= 150 and 10 <= b <= 60:
                x_ref, y_ref = j, i
                img_ref = get_ref_image(img, x_ref, y_ref)
                return x_ref, y_ref, img_ref, h
    
    return x_ref, y_ref + 710, img, h

def get_ref_image(img, x_ref, y_ref):
    #creates green dot indication of reference frame on background image
    x_ref_upper = x_ref + 4
    y_ref_upper = y_ref + 4
    if x_ref < 3:
        x_ref_lower = 0
    else:
        x_ref_lower = x_ref - 3
    if y_ref < 3:
        y_ref_lower = 0
    else:
        y_ref_lower = y_ref - 3
    pix = img.load()
    for x in range(x_ref_lower, x_ref_upper):
        for y in range(y_ref_lower, y_ref_upper):
            pix[x,y] = (0, 255, 0)
    
    return img    

if __name__ == '__main__':
    img_path = '/mnt/c334c9bc-7ae4-4ea7-84fb-6b8f5595aea2/overhead/'
    csv_path = '/mnt/c334c9bc-7ae4-4ea7-84fb-6b8f5595aea2/overhead/overhead.csv'
    background_path = '/mnt/c334c9bc-7ae4-4ea7-84fb-6b8f5595aea2/overhead/background.jpeg'
    df = pandas.read_csv(csv_path, encoding = 'utf-8')
    X = []
    Y = []
    YAW = []
    x = 0
    y = 0
    yaw = 0
    counter = 0

    #get background 
    background = cv2.imread(background_path)
    background2 = background[:, 240:1150]
    
    #get reference coordinates for reference frame
    x_ref, y_ref, img_ref, h = get_ref_coordinates(background_path)
    
    #make thresh, make white transparent, overlay with original, track color (frameDelta)
    
    #iterate through all frames
    for i, row in tqdm(df.iterrows()):
        fname = row['file_location']
        #Time extracted from csv file
        ftime = row['time']
        print('iteration')
        interval = get_interval(ftime)
      
        #img is modified with boxes and added text
        img = cv2.imread(os.path.join(img_path, fname[:-1]))[:, 240:1150]
        orig_img = img.copy()
        orig_img2 = img.copy()
        #subtract background
        background = cv2.GaussianBlur(background2, (17, 17), 0)
        img = cv2.GaussianBlur(img, (17, 17), 0)

        frameDelta = cv2.absdiff(background, img)
        
        #Green
        frameDelta[:, :, 1] = 0

        #Red
        for a in range(720):
            for b in range(910):
                 if frameDelta[a, b, 0] < 25:
                        frameDelta[a,b,0] = 0 
        #Blue
        for a in range(720):
            for b in range(910):
                 if frameDelta[a, b, 2] < 25:
                        frameDelta[a,b,2] = 0       
        


        frameDelta = cv2.cvtColor(frameDelta,cv2.COLOR_BGR2RGB)
        
        #track with framDelta
        x, y, yaw, orig_img2, v1_prev_velocity, v1_x_prev_ref, v1_y_prev_ref, img_overlay \
            = get_coordinates(orig_img2, interval, frameDelta, v1_prev_velocity, v1_x_prev_ref, v1_y_prev_ref, 
                              x, y, x_ref, y_ref, v1_lower_bound, v1_upper_bound, "RED", bgr_red, 170, orig_img)
        
        x, y, yaw, orig_img2, v2_prev_velocity, v2_x_prev_ref, v2_y_prev_ref, img_overlay \
            = get_coordinates(orig_img2, interval, frameDelta, v2_prev_velocity, v2_x_prev_ref, v2_y_prev_ref, 
                              x, y, x_ref, y_ref, v2_lower_bound, v2_upper_bound, "BLUE", bgr_blue, 370, orig_img)
        
        #x, y, yaw, img, v3_prev_velocity, v3_x_prev_ref, v3_y_prev_ref, img_overlay \
        #    = get_coordinates(img, interval, frameDelta, v3_prev_velocity, v3_x_prev_ref, v3_y_prev_ref, 
        #                      x, y, x_ref, y_ref, v3_lower_bound, v3_upper_bound, "GREEN", bgr_green, 800, img_overlay)
        
        cv2.addWeighted(img_overlay, 0.25, orig_img2, 1, 0, img)
        
        # Resize to less than 640 and 480 pixel lengths respectively
        video_img = cv2.resize(img, (1280, 720), interpolation = cv2.INTER_AREA)
        #cv2.imshow('img', video_img)
        #cv2.waitKey(0)
        video.write(video_img)
        
        
    video.release()
    cv2.destroyAllWindows()
    csv_input = pandas.read_csv(csv_path)
    #csv_input['Position_X'] = X
    #csv_input['Position_Y'] = Y
    #csv_input['Yaw'] = YAW
    #csv_input.to_csv('/home/jesse/Desktop/Logitech_Brio/output.csv', index=False)
    fraction = 0.3
    colors =  [plt.cm.rainbow(i) for i in np.linspace(0, 1, (counter))]
    ys = [i for i in range(int((counter-1)*fraction))]
    for y, c in zip(ys, colors):
        plt.scatter(X, Y, c=colors, s=2)
    print('Done')






0it [00:00, ?it/s]

4.5.1
iteration
38.692472





1it [00:01,  1.57s/it]

iteration
38.701690





2it [00:03,  1.55s/it]

...

iteration
02.514433





1991it [50:21,  1.49s/it]

iteration
02.527786





1992it [50:22,  1.52s/it]

Done
